# 핵심이슈 키워드 추출 후 월별 그룹화 파일 자동 추출

- 핵심이슈 키워드를 TF-IDF로 추출
- 추출한 데이터의 연도를 입력하면 해당 년도의 `이슈 키워드`를 1월부터 12월까지 `월별`로 추출을 시작함.

In [10]:
import json
import os
import re
from tqdm import tqdm
from collections import Counter, defaultdict
from datetime import datetime, timedelta
from konlpy.tag import Okt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import calendar

# =========================
# 설정
# =========================
file_path = '/home/ds4_sia_nolb/#FINAL_POLARIS/04_plus_preprocessing/preprocessing_final_data/re_final_preprocessing.json'
output_dir = '/home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/re_monthly_results'
TFIDF_VECTORIZER_PATH = '/home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/re_idf_vectorizer_for_all_corpus.pkl'

# 결과 저장 디렉토리 생성
os.makedirs(output_dir, exist_ok=True)

# =========================
# 형태소 분석기
# =========================
okt = Okt()

# =========================
# 날짜 파서 (여러 포맷 허용)
# =========================
def parse_date_flexible(s: str):
    if not s or not isinstance(s, str):
        return None
    s = s.strip()

    candidates = [s]
    if "T" in s:
        candidates.append(s[:19])
        candidates.append(s[:10])
    if len(s) >= 10:
        candidates.append(s[:10])
    if "-" not in s and "." not in s and "/" not in s and len(s) == 8:
        candidates.append(f"{s[:4]}-{s[4:6]}-{s[6:8]}")

    fmts = [
        "%Y-%m-%d",
        "%Y-%m-%d %H:%M:%S",
        "%Y-%m-%d %H:%M",
        "%Y/%m/%d",
        "%Y/%m/%d %H:%M:%S",
        "%Y.%m.%d",
        "%Y.%m.%d %H:%M:%S",
        "%Y.%m.%d %H:%M",
        "%Y%m%d",
        "%Y-%m-%dT%H:%M:%S",
    ]

    for cand in candidates:
        for fmt in fmts:
            try:
                return datetime.strptime(cand, fmt)
            except Exception:
                pass
    return None

def parse_date(date_str: str) -> datetime:
    d = parse_date_flexible(date_str)
    if d is None:
        raise ValueError(f"날짜 형식이 올바르지 않습니다: {date_str}")
    return d

def extract_pubdate(article):
    keys = ["pubDate", "pubdate", "time", "date", "published", "pub_date"]
    for k in keys:
        if k in article and article[k]:
            dt = parse_date_flexible(str(article[k]))
            if dt:
                return dt
    meta = article.get("metadata", {}) or {}
    for k in keys:
        if k in meta and meta[k]:
            dt = parse_date_flexible(str(meta[k]))
            if dt:
                return dt
    return None

# =========================
# 기사 로드 및 필터링 유틸
# =========================
def load_all_articles(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        if not isinstance(data, list):
            raise ValueError("JSON 루트는 list 여야 합니다.")
        return data
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
        return None
    except json.JSONDecodeError as e:
        print(f"JSON 파싱 오류: {e}")
        return None
    except Exception as e:
        print(f"알 수 없는 오류가 발생했습니다: {e}")
        return None

def filter_articles_by_period(articles, start_date_str, end_date_str):
    sdt = parse_date(start_date_str)
    edt = parse_date(end_date_str)
    
    period_articles = []
    
    all_articles_count = len(articles)
    
    for article in tqdm(articles, desc=f"기사 처리 중 ({sdt.date()}~{edt.date()})", leave=False):
        pub_date = extract_pubdate(article)
        if pub_date and sdt <= pub_date <= edt:
            period_articles.append(article)
    
    print(f"총 {all_articles_count}개의 기사 중 지정 기간 내 기사 {len(period_articles)}개를 찾았습니다.")
    
    return period_articles

# =========================
# 텍스트 정규화 (표기 통일 약간)
# =========================
def normalize_text(t: str) -> str:
    if not t:
        return ""
    t = t.replace("탄도 미사일", "탄도미사일")
    t = t.replace("순항 미사일", "순항미사일")
    t = t.replace("극초 음속", "극초음속")
    t = t.replace("초대형 방사포", "초대형방사포")
    return t

# =========================
# 불용어 & 뉴스 노이즈
# =========================
BASE_STOP = set([
    '가','간','같은','같이','것','게다가','결국','곧','관하여','관련','관한','그','그것','그녀','그들',
    '그리고','그때','그래','그래서','그러나','그러므로','그러한','그런','그렇게','그외','근거로','기타',
    '까지도','까지','나','남들','너','누구','다','다가','다른','다만','다소','다수','다시','다음','단','단지',
    '당신','대','대해서','더군다나','더구나','더라도','더욱이','도','도로','또','또는','또한','때','때문',
    '라도','라면','라는','로','로부터','로써','를','마저','마치','만약','만일','만큼','모두','무엇','무슨',
    '무척','물론','및','밖에','바로','보다','뿐이다','사람','사실은','상대적으로','생각','설령','소위','수',
    '수준','쉽게','시대','시작하여','실로','실제','아니','아무','아무도','아무리','아마도','아울러','아직',
    '앞에서','앞으로','어느','어떤','어떻게','어디','언제','얼마나','여기','여부','역시','예','오히려',
    '와','왜','외에도','요','우리','우선','원래','위해서','으로','으로부터','으로써','을','의','의거하여',
    '의지하여','의해','의해서','의하여','이','이것','이곳','이때','이라고','이러한','이런','이렇게','이제',
    '이지만','이후','이상','이다','이전','인','일','일단','일반적으로','임시로','입장에서','자','자기','자신',
    '잠시','저','저것','저기','저쪽','저희','전부','전혀','점에서','정도','제','조금','좀','주로','주제','즉',
    '즉시','지금','진짜로','차라리','참','참으로','첫번째로','최고','최대','최소','최신','최초','통하여',
    '통해서','평가','포함한','포함하여','하지만','하면서','하여','한','한때','한번','할','할것이다','할수있다',
    '함께','해도', 
    # 아래 키워드는 idf_vectorizer_for_all_corpus.pkl파일 생성 이후 추가된 불용어임. BASE_STOP에 있으면 pkl파일로 인해 미적용 되기 때문에 ENTITY_NOISE에 추가하였음.
    # '돼다', '서다', '대해', '나오다', '통해', '맞다', '대한', '위해', '기상청', '예보', '밝히다', '크다', '약간', '가다', '내리다', '받다', '기온'
])

NEWS_STOP = {"기자","연합뉴스","사진","속보","종합","자료","영상","단독","전문","인터뷰","브리핑"}

# =========================
# 엔터티 노이즈
# =========================
ENTITY_NOISE = {
    "북한","한국","대한민국","남한","미국","중국","일본","러시아","우크라이나","유엔","나토","NATO","EU","유럽연합",
    "푸틴","블라디미르 푸틴","바이든","조 바이든","시진핑","김정은","김여정","문재인","윤석열","쇼이구","젤렌스키", "중앙", "통신", "보도", 
    # 아래 키워드는 BASE_STOP에 있어야하지만 잠시 옮겨옴.
    '돼다', '서다', '대해', '나오다', '통해', '맞다', '대한', '위해', '기상청', '예보', '밝히다', '크다', '약간', '가다', '내리다', '받다', '기온',
    '강수', '날씨', '소식통', '인용', '대체로', '이번', '들다', '들어', '올해', 

}

# =========================
# 토큰/텍스트
# =========================
def pos_tokens(text: str):
    text = normalize_text(text or "")
    return okt.pos(text, norm=True, stem=True)

def doc_text(a) -> str:
    # 수정: metadata의 title과 최상위 summary를 함께 사용
    title = (a.get('metadata') or {}).get('title', '')
    summary = a.get('summary', '')
    return normalize_text(f"{title} {summary}")

def tokenizer_for_vectorizer(s: str):
    toks = []
    for w, t in okt.pos(s, norm=True, stem=True):
        if t not in ("Noun", "Verb"):
            continue
        if len(w) <= 1:
            continue
        if w in BASE_STOP or w in NEWS_STOP:
            continue
        if w.isdigit():
            continue
        toks.append(w)
    return toks

# =========================
# 자동 학습: '행동 동사'와 '행위 명사'
# =========================
def learn_action_lexicons(articles, min_df_ratio_verbs=0.002, min_df_ratio_nouns=0.002):
    verb_doc_df = Counter()
    action_noun_df = Counter()
    N_docs = len(articles)

    for a in tqdm(articles, desc="행동 동사/행위명사 학습 중", leave=False):
        title = (a.get('metadata') or {}).get('title', '') # 수정
        summary = a.get('summary','') or ''
        p = pos_tokens(f"{title} {summary}")

        verbs_in_doc = set()
        action_nouns_in_doc = set()

        for i, (w, t) in enumerate(p):
            if t == "Verb":
                verbs_in_doc.add(w)
            if t == "Noun":
                ahead = [p[j][0] for j in range(i+1, min(i+3, len(p)))]
                if "하다" in ahead or "되다" in ahead:
                    if w not in BASE_STOP and len(w) > 1:
                        action_nouns_in_doc.add(w)

        for v in verbs_in_doc:
            verb_doc_df[v] += 1
        for n in action_nouns_in_doc:
            action_noun_df[n] += 1

    min_df_verbs = max(5, int(N_docs * min_df_ratio_verbs))
    min_df_nouns = max(5, int(N_docs * min_df_ratio_nouns))

    drop_verbs = {"하다","되다","이다","있다"}
    verb_set = {v for v,df in verb_doc_df.items() if df >= min_df_verbs and v not in drop_verbs}
    action_nouns = {n for n,df in action_noun_df.items() if df >= min_df_nouns}

    print(f"학습 결과: 행동동사 {len(verb_set)}개, 행위명사 {len(action_nouns)}개")
    return verb_set, action_nouns

# =========================
# 사건 구 후보 생성 + TF-IDF 결합 랭킹
# =========================
def nominalize_verb(v: str) -> str:
    if v.endswith("하다"):
        return v[:-2]
    if v.endswith("되다"):
        return v[:-2]
    return v

def extract_event_phrases_auto(articles, top_k=30, vectorizer=None):
    N = len(articles)
    if N == 0:
        print("⚠ 지정 기간에 기사가 없습니다.")
        return []

    print(f"지정 기간 내 기사 수: {N}개")
    
    verb_set, action_nouns = learn_action_lexicons(articles)

    phrase_df = Counter()
    phrase_examples = defaultdict(list)

    for a in tqdm(articles, desc="사건 구 자동 추출 중", leave=False):
        # 수정: metadata에서 title을 가져옴
        title = (a.get('metadata') or {}).get('title', '')
        summary = a.get('summary','') or ''
        p = pos_tokens(f"{title} {summary}")
        phrases_in_doc = set()

        prev_nouns = []
        L = len(p)
        for i, (w, t) in enumerate(p):
            if t == "Noun":
                if w not in BASE_STOP and len(w) > 1:
                    prev_nouns.append(w)
                    if len(prev_nouns) > 5:
                        prev_nouns = prev_nouns[-5:]

            if t == "Verb" and w in verb_set:
                vnom = nominalize_verb(w)
                nn = [n for n in reversed(prev_nouns)][:2]
                if nn:
                    phrases_in_doc.add(f"{nn[0]} {vnom}".strip())
                    if len(nn) >= 2:
                        phrases_in_doc.add(f"{nn[1]} {nn[0]} {vnom}".strip())
                else:
                    phrases_in_doc.add(vnom.strip())

            if t == "Noun" and w in action_nouns:
                nn = [n for n in reversed(prev_nouns) if n != w][:2]
                base = w
                if nn:
                    phrases_in_doc.add(f"{nn[0]} {base}".strip())
                    if len(nn) >= 2:
                        phrases_in_doc.add(f"{nn[1]} {nn[0]} {base}".strip())
                else:
                    phrases_in_doc.add(base.strip())

                if i+1 < L and p[i+1][1] == "Noun" and p[i+1][0] in action_nouns:
                    tail = p[i+1][0]
                    if nn:
                        phrases_in_doc.add(f"{nn[0]} {base} {tail}".strip())
                        if len(nn) >= 2:
                            phrases_in_doc.add(f"{nn[1]} {nn[0]} {base} {tail}".strip())
                    else:
                        phrases_in_doc.add(f"{base} {tail}".strip())

        cleaned = set()
        for ph in phrases_in_doc:
            ph = re.sub(r"\s+", " ", ph).strip()
            if len(ph.split()) == 1 and len(ph) <= 2:
                continue
            cleaned.add(ph)

        for ph in cleaned:
            phrase_df[ph] += 1
            # 수정: title이 존재할 때만 examples에 추가
            if len(phrase_examples[ph]) < 3 and title:
                phrase_examples[ph].append(title)

    if vectorizer is None:
        print("[오류] TfidfVectorizer 객체가 전달되지 않았습니다.")
        return []

    corpus_period = [doc_text(a) for a in articles]
    Xp = vectorizer.transform(corpus_period)
    tfidf_avg = np.asarray(Xp.mean(axis=0)).ravel()
    terms = vectorizer.get_feature_names_out()
    tfidf_dict = {terms[i]: float(tfidf_avg[i]) for i in np.where(tfidf_avg > 0)[0]}

    print(f"TF-IDF 코퍼스: {len(corpus_period)}개 문서")
    print(f"TF-IDF 용어수: {len(terms)}")

    def is_entity_only(ph: str) -> bool:
        toks = ph.split()
        if len(toks) <= 2 and any(ent in ph for ent in ENTITY_NOISE):
            return True
        ent_hits = sum(1 for t in toks if any(ent in t for ent in ENTITY_NOISE))
        return (ent_hits >= max(1, len(toks) - 1))

    def generic_penalty(ph: str) -> int:
        generic = {"대통령","위원장","정부","당국","관계자","대변인","회의","논의","강조"}
        return -sum(1 for t in ph.split() if t in generic)

    def phrase_score(ph: str, df_cnt: int) -> float:
        tfidf = tfidf_dict.get(ph, 0.0)
        score = 0.6 * tfidf + 0.4 * float(df_cnt)

        if is_entity_only(ph):
            score -= 6.0
        score += generic_penalty(ph)
        if len(ph.split()) <= 2:
            score -= 1.5
        return score

    scored = []
    for ph, cnt in phrase_df.items():
        if is_entity_only(ph):
            continue
        scored.append( (ph, cnt, phrase_score(ph, cnt)) )

    scored.sort(key=lambda x: (x[2], x[1]), reverse=True)
    ranked = [(ph, cnt, score, phrase_examples.get(ph, [])) for ph, cnt, score in scored[:top_k]]
    return ranked

# =========================
# 전체 코퍼스용 TF-IDF 벡터라이저 사전 학습
# =========================
def pre_train_vectorizer(articles, save_path):
    if os.path.exists(save_path):
        print(f"✔️ 기존 TF-IDF 벡터라이저 파일 '{save_path}'이 이미 존재합니다. 학습을 건너뜁니다.")
        return joblib.load(save_path)
    
    print(f"🔍 전체 코퍼스용 TF-IDF 벡터라이저를 새로 학습합니다.")
    
    full_corpus = [doc_text(a) for a in articles]
    
    vectorizer = TfidfVectorizer(
        tokenizer=tokenizer_for_vectorizer,
        ngram_range=(1, 3),
        min_df=5,
        max_df=0.85,
        sublinear_tf=True,
        norm='l2'
    )
    vectorizer.fit(full_corpus)
    joblib.dump(vectorizer, save_path)
    print(f"✅ 전체 코퍼스 기반 TF-IDF 벡터라이저를 '{save_path}'에 저장했습니다.")
    return vectorizer

# =========================
# 월별 자동 처리 함수
# =========================
def process_monthly_keywords(year, all_articles, vectorizer):
    """지정된 연도의 모든 월에 대해 키워드를 추출하고 결과를 저장합니다."""
    
    results = {}
    
    for month in range(1, 13):
        print(f"\n{'='*50}")
        print(f"📅 {year}년 {month}월 키워드 추출 중...")
        print(f"{'='*50}")
        
        start_date = f"{year}-{month:02d}-01"
        last_day = calendar.monthrange(year, month)[1]
        end_date = f"{year}-{month:02d}-{last_day:02d}"
        
        monthly_articles = filter_articles_by_period(all_articles, start_date, end_date)
        
        if not monthly_articles:
            print(f"⚠️ {year}년 {month}월에 기사가 없습니다.")
            results[f"{year}_{month:02d}"] = []
            continue
        
        events = extract_event_phrases_auto(
            monthly_articles,
            top_k=30,
            vectorizer=vectorizer
        )
        
        results[f"{year}_{month:02d}"] = events
        
        print(f"\n=== {year}년 {month}월 '사건 구' TOP 30 (TF-IDF + DF 결합) ===")
        if not events:
            print(f"{year}년 {month}월에 추출된 사건 구가 없습니다.")
        else:
            for i, (ph, cnt, score, examples) in enumerate(events[:10], 1):
                print(f"{i}. {ph}   (점수={score:.2f}, 문서수={cnt})")
        
        output_file = os.path.join(output_dir, f"{year}_{month:02d}_keywords.json")
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump({
                'year': year,
                'month': month,
                'period': f"{start_date} ~ {end_date}",
                'total_articles': len(monthly_articles),
                'keywords': [{'phrase': ph, 'doc_count': cnt, 'score': round(score, 2), 'examples': examples} 
                             for ph, cnt, score, examples in events]
            }, f, ensure_ascii=False, indent=2)
        
        print(f"💾 결과가 '{output_file}'에 저장되었습니다.")
    
    return results

# =========================
# 실행부
# =========================
if __name__ == '__main__':
    try:
        print("📖 전체 기사 데이터를 로드하는 중...")
        all_articles = load_all_articles(file_path)
        if not all_articles:
            print("전체 코퍼스를 로드할 수 없습니다. 프로그램을 종료합니다.")
            exit()

        vectorizer = pre_train_vectorizer(all_articles, TFIDF_VECTORIZER_PATH)
        
        year = int(input("분석할 연도를 입력하세요 (예: 2024): ").strip())
        
        print(f"\n🚀 {year}년 월별 키워드 추출을 시작합니다...")
        
        monthly_results = process_monthly_keywords(year, all_articles, vectorizer)
        
        summary_file = os.path.join(output_dir, f"{year}_keywords_by_month_all.json")
        with open(summary_file, 'w', encoding='utf-8') as f:
            json.dump(monthly_results, f, ensure_ascii=False, indent=2)
        
        print(f"\n🎉 {year}년 월별 키워드 추출이 완료되었습니다!")
        print(f"📁 결과 파일들이 '{output_dir}' 디렉토리에 저장되었습니다.")
        print(f"📊 연간 종합 결과: '{summary_file}'")
        
    except ValueError as e:
        print(f"오류: {e}. 올바른 연도를 입력해주세요.")
    except Exception as e:
        print(f"예기치 않은 오류가 발생했습니다: {e}")

📖 전체 기사 데이터를 로드하는 중...
✔️ 기존 TF-IDF 벡터라이저 파일 '/home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/re_idf_vectorizer_for_all_corpus.pkl'이 이미 존재합니다. 학습을 건너뜁니다.

🚀 2025년 월별 키워드 추출을 시작합니다...

📅 2025년 1월 키워드 추출 중...


총 80434개의 기사 중 지정 기간 내 기사 385개를 찾았습니다.
지정 기간 내 기사 수: 385개


학습 결과: 행동동사 109개, 행위명사 99개


TF-IDF 코퍼스: 385개 문서
TF-IDF 용어수: 359320

=== 2025년 1월 '사건 구' TOP 30 (TF-IDF + DF 결합) ===
1. 탄도미사일 발사   (점수=16.11, 문서수=44)
2. 북한 탄도미사일 발사   (점수=10.80, 문서수=27)
3. 시험 발사   (점수=7.70, 문서수=23)
4. 북한 군인 생포   (점수=6.80, 문서수=17)
5. 동해 발사   (점수=6.50, 문서수=20)
6. 미사일 발사   (점수=6.50, 문서수=20)
7. 영상 공개   (점수=6.50, 문서수=20)
8. 군인 생포 밝히다   (점수=6.00, 문서수=15)
9. 일대 동해 발사   (점수=5.60, 문서수=14)
10. 단거리 탄도미사일 발사   (점수=5.60, 문서수=14)
💾 결과가 '/home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/re_monthly_results/2025_01_keywords.json'에 저장되었습니다.

📅 2025년 2월 키워드 추출 중...


총 80434개의 기사 중 지정 기간 내 기사 298개를 찾았습니다.
지정 기간 내 기사 수: 298개


...

총 80434개의 기사 중 지정 기간 내 기사 0개를 찾았습니다.
⚠️ 2025년 8월에 기사가 없습니다.

📅 2025년 9월 키워드 추출 중...


총 80434개의 기사 중 지정 기간 내 기사 0개를 찾았습니다.
⚠️ 2025년 9월에 기사가 없습니다.

📅 2025년 10월 키워드 추출 중...


총 80434개의 기사 중 지정 기간 내 기사 0개를 찾았습니다.
⚠️ 2025년 10월에 기사가 없습니다.

📅 2025년 11월 키워드 추출 중...


총 80434개의 기사 중 지정 기간 내 기사 0개를 찾았습니다.
⚠️ 2025년 11월에 기사가 없습니다.

📅 2025년 12월 키워드 추출 중...


총 80434개의 기사 중 지정 기간 내 기사 0개를 찾았습니다.
⚠️ 2025년 12월에 기사가 없습니다.

🎉 2025년 월별 키워드 추출이 완료되었습니다!
📁 결과 파일들이 '/home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/re_monthly_results' 디렉토리에 저장되었습니다.
📊 연간 종합 결과: '/home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/re_monthly_results/2025_keywords_by_month_all.json'


# 키워드 클러스터

- 월별 키워드 추출했을 때 의미가 중복되는 키워드가 다수 보임.
- 해당 문제는 키워드별 언급된 기사 수(doc_count)와 그에 따른 순위 점수(score)가 분산되어 해당 키워드의 중요도가 낮아질 문제가 있음.
- 그래서 비슷한 의미의 키워드를 병합하는 코드를 추가로 만듬.
- 코드 하단에 연도, 연도별 특정 월 등 원하는 방법에 따라 약간의 변경 후 실행하면 추출되어 있던 키워드들을 병합해서 json 파일로 저장하게 됨.
- similarity_threshold 에 따라서 키워드의 유사도를 결정할 수 있음.

In [21]:
import json
import math
import os
from typing import List, Dict, Tuple, Any
from collections import Counter
import itertools

class KeywordGrouper:
    """유사도 기반 키워드 자동 그룹화 클래스 - 키워드 보너스 제거 버전"""
    
    def __init__(self):
        # important_keywords 제거 - 순수 문자열 유사도만 사용
        pass
    
    def jaccard_similarity(self, str1: str, str2: str) -> float:
        """자카드 유사도 계산 (단어 집합 기반)"""
        set1 = set(str1.split())
        set2 = set(str2.split())
        
        intersection = set1.intersection(set2)
        union = set1.union(set2)
        
        return len(intersection) / len(union) if len(union) > 0 else 0
    
    def levenshtein_distance(self, str1: str, str2: str) -> float:
        """편집 거리 기반 유사도 (레벤슈타인 거리)"""
        if len(str1) == 0:
            return len(str2)
        if len(str2) == 0:
            return len(str1)
        
        # 동적 프로그래밍 매트릭스 생성
        matrix = [[0] * (len(str1) + 1) for _ in range(len(str2) + 1)]
        
        # 첫 번째 행과 열 초기화
        for i in range(len(str2) + 1):
            matrix[i][0] = i
        for j in range(len(str1) + 1):
            matrix[0][j] = j
        
        # 매트릭스 채우기
        for i in range(1, len(str2) + 1):
            for j in range(1, len(str1) + 1):
                if str2[i-1] == str1[j-1]:
                    matrix[i][j] = matrix[i-1][j-1]
                else:
                    matrix[i][j] = min(
                        matrix[i-1][j-1] + 1,  # substitution
                        matrix[i][j-1] + 1,    # insertion
                        matrix[i-1][j] + 1     # deletion
                    )
        
        max_len = max(len(str1), len(str2))
        return 1 - (matrix[len(str2)][len(str1)] / max_len) if max_len > 0 else 1
    
    def calculate_similarity(self, phrase1: str, phrase2: str) -> float:
        """순수 문자열 유사도 계산 - 키워드 보너스 제거"""
        # 자카드 유사도 (단어 겹침)
        jaccard_score = self.jaccard_similarity(phrase1, phrase2)
        
        # 편집 거리 유사도 (문자열 유사성)
        levenshtein_score = self.levenshtein_distance(phrase1, phrase2)
        
        # 가중 평균 (자카드 70%, 편집거리 30%)
        return jaccard_score * 0.7 + levenshtein_score * 0.3
    
    def auto_group_keywords(self, keywords: List[Dict], similarity_threshold: float = 0.4, 
                           min_group_size: int = 2) -> List[Dict]:
        """클러스터링을 통한 자동 그룹화"""
        groups = []
        processed = set()
        
        for i in range(len(keywords)):
            if i in processed:
                continue
            
            current_group = [keywords[i]]
            current_group_indices = [i]
            processed.add(i)
            
            # 현재 키워드와 유사한 키워드들 찾기
            for j in range(i + 1, len(keywords)):
                if j in processed:
                    continue
                
                similarity = self.calculate_similarity(
                    keywords[i]['phrase'], 
                    keywords[j]['phrase']
                )
                
                if similarity >= similarity_threshold:
                    current_group.append(keywords[j])
                    current_group_indices.append(j)
                    processed.add(j)
            
            # 그룹 정보 저장
            groups.append({
                'keywords': current_group,
                'indices': current_group_indices,
                'avg_similarity': self.calculate_group_average_similarity(current_group)
            })
        
        return groups
    
    def calculate_group_average_similarity(self, group_keywords: List[Dict]) -> float:
        """그룹 내 평균 유사도 계산"""
        if len(group_keywords) < 2:
            return 1.0
        
        total_similarity = 0
        pair_count = 0
        
        for i in range(len(group_keywords)):
            for j in range(i + 1, len(group_keywords)):
                total_similarity += self.calculate_similarity(
                    group_keywords[i]['phrase'],
                    group_keywords[j]['phrase']
                )
                pair_count += 1
        
        return total_similarity / pair_count if pair_count > 0 else 1.0
    
    def select_representative_keyword(self, group_keywords: List[Dict]) -> Dict:
        """그룹 대표 키워드 선정 (가장 높은 점수)"""
        return max(group_keywords, key=lambda k: k['score'])
    
    def find_common_words(self, phrases: List[str]) -> List[str]:
        """공통 단어 찾기"""
        word_counts = Counter()
        min_occurrence = math.ceil(len(phrases) * 0.6)  # 60% 이상 출현
        
        for phrase in phrases:
            words = phrase.split()
            for word in words:
                word_counts[word] += 1
        
        common_words = [word for word, count in word_counts.items() 
                       if count >= min_occurrence]
        
        # 빈도순으로 정렬
        return sorted(common_words, key=lambda w: word_counts[w], reverse=True)
    
    def generate_group_name(self, group_keywords: List[Dict]) -> str:
        """그룹명 생성 - 원래 단어 순서 유지"""
        representative = self.select_representative_keyword(group_keywords)
        phrases = [k['phrase'] for k in group_keywords]
        
        # 공통 단어 추출
        common_words = self.find_common_words(phrases)
        
        if common_words and len(common_words) > 1:
            # 대표 키워드에서 공통 단어들의 순서 찾기
            rep_words = representative['phrase'].split()
            ordered_common = []
            
            # 대표 키워드의 단어 순서대로 공통 단어 배열
            for word in rep_words:
                if word in common_words:
                    ordered_common.append(word)
            
            # 빠진 공통 단어들 추가
            for word in common_words:
                if word not in ordered_common:
                    ordered_common.append(word)
            
            if ordered_common:
                return ' '.join(ordered_common)
        
        # 공통 단어가 없거나 1개면 대표 키워드 사용
        return representative['phrase']
    
    def smart_group_keywords(self, keyword_data: Dict, **options) -> Dict:
        """메인 그룹화 함수"""
        # 기본 옵션 설정
        similarity_threshold = options.get('similarity_threshold', 0.4)
        min_group_size = options.get('min_group_size', 2)
        max_groups = options.get('max_groups', 15)
        
        print("=== 자동 키워드 그룹화 시작 (키워드 보너스 제거 버전) ===")
        print(f"원본 키워드 수: {len(keyword_data['keywords'])}")
        print(f"유사도 임계값: {similarity_threshold}")
        print(f"최소 그룹 크기: {min_group_size}")
        print(f"유사도 계산: 자카드(70%) + 편집거리(30%)")
        
        # 자동 그룹화 수행
        groups = self.auto_group_keywords(
            keyword_data['keywords'], 
            similarity_threshold, 
            min_group_size
        )
        
        # 그룹 정보를 최종 형태로 변환
        grouped_keywords = []
        
        for group in groups:
            representative = self.select_representative_keyword(group['keywords'])
            group_name = self.generate_group_name(group['keywords'])
            
            # 예시 문장 중복 제거
            all_examples = []
            for keyword in group['keywords']:
                all_examples.extend(keyword['examples'])
            unique_examples = list(dict.fromkeys(all_examples))[:3]
            
            grouped_keyword = {
                'phrase': group_name,
                'doc_count': sum(k['doc_count'] for k in group['keywords']),
                'score': round(sum(k['score'] for k in group['keywords']), 1),
                'examples': unique_examples,
                'merged_keywords': [k['phrase'] for k in group['keywords']],
                'keyword_count': len(group['keywords']),
                'avg_similarity': round(group['avg_similarity'], 3),
                'representative_keyword': representative['phrase']
            }
            
            grouped_keywords.append(grouped_keyword)
        
        # 점수순 정렬
        grouped_keywords.sort(key=lambda x: x['score'], reverse=True)
        
        # 최대 그룹 수 제한
        final_keywords = grouped_keywords[:max_groups]
        
        # 결과 출력
        print(f"\n=== 그룹화 완료 ===")
        print(f"최종 키워드 그룹 수: {len(final_keywords)}")
        compression_rate = round((1 - len(final_keywords) / len(keyword_data['keywords'])) * 100, 1)
        print(f"압축률: {compression_rate}%")
        
        # 상위 그룹들 출력
        print("\n=== 상위 그룹들 (순수 문자열 유사도 기반) ===")
        for i, group in enumerate(final_keywords[:8], 1):
            print(f"{i}. {group['phrase']} (점수: {group['score']})")
            print(f"   - 통합된 키워드 수: {group['keyword_count']}")
            print(f"   - 평균 유사도: {group['avg_similarity']}")
            print(f"   - 통합 키워드: {', '.join(group['merged_keywords'])}")
            print()
        
        return {
            'year': keyword_data['year'],
            'month': keyword_data['month'],
            'period': keyword_data['period'],
            'total_articles': keyword_data['total_articles'],
            'original_keyword_count': len(keyword_data['keywords']),
            'grouped_keyword_count': len(final_keywords),
            'compression_rate': compression_rate,
            'similarity_threshold': similarity_threshold,
            'similarity_method': 'jaccard(70%) + levenshtein(30%)',
            'keywords': final_keywords
        }
    
    def test_different_thresholds(self, keyword_data: Dict) -> None:
        """다양한 임계값으로 테스트"""
        thresholds = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
        
        print("=== 다양한 유사도 임계값 테스트 (키워드 보너스 제거 버전) ===")
        for threshold in thresholds:
            result = self.smart_group_keywords(
                keyword_data, 
                similarity_threshold=threshold,
                min_group_size=2
            )
            
            print(f"임계값 {threshold}: {result['original_keyword_count']} → "
                  f"{result['grouped_keyword_count']} ({result['compression_rate']}% 압축)")
        print()
    
    def load_json_file(self, file_path: str) -> Dict:
        """JSON 파일 로드"""
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                return json.load(f)
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {file_path}")
            return None
        except json.JSONDecodeError:
            print(f"JSON 파싱 오류: {file_path}")
            return None
    
    def save_grouped_results(self, result: Dict, output_path: str) -> None:
        """그룹화 결과를 JSON 파일로 저장"""
        try:
            # 출력 디렉토리가 없으면 생성
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(result, f, ensure_ascii=False, indent=2)
            print(f"결과가 저장되었습니다: {output_path}")
        except Exception as e:
            print(f"저장 중 오류 발생: {e}")


# 자동화된 배치 처리 함수들
def process_single_file(grouper: KeywordGrouper, year: int, month: int, base_input_dir: str, base_output_dir: str, **options):
    """단일 파일 처리"""
    # 입력 파일 경로 생성
    input_file = os.path.join(base_input_dir, f"{year}_{month:02d}_keywords.json")
    
    # 출력 파일 경로 생성
    output_file = os.path.join(base_output_dir, f"{year}_{month:02d}_keyword_grouped.json")
    
    print(f"\n{'='*60}")
    print(f"처리 중: {year}년 {month}월")
    print(f"입력 파일: {input_file}")
    print(f"출력 파일: {output_file}")
    print(f"{'='*60}")
    
    # 파일 존재 여부 확인
    if not os.path.exists(input_file):
        print(f"❌ 입력 파일이 존재하지 않습니다: {input_file}")
        return False
    
    try:
        # 파일 로드
        data = grouper.load_json_file(input_file)
        if data is None:
            return False
        
        # 그룹화 수행
        result = grouper.smart_group_keywords(data, **options)
        
        # 결과 저장
        grouper.save_grouped_results(result, output_file)
        
        print(f"✅ 성공적으로 처리되었습니다: {year}년 {month}월")
        return True
        
    except Exception as e:
        print(f"❌ 처리 중 오류 발생: {e}")
        return False


def process_year_batch(year: int, months: List[int] = None, **options):
    """연간 배치 처리"""
    if months is None:
        months = list(range(1, 13))  # 1월부터 12월까지
    
    # 기본 경로 설정
    base_input_dir = f'/home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/re_monthly_results'
    base_output_dir = f'/home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/re_monthly_results_cluster'
    
    # 사용자 정의 경로가 있으면 적용
    base_input_dir = options.pop('input_dir', base_input_dir)
    base_output_dir = options.pop('output_dir', base_output_dir)
    
    grouper = KeywordGrouper()
    
    print(f"🚀 {year}년 키워드 그룹화 배치 처리 시작")
    print(f"📂 입력 디렉토리: {base_input_dir}")
    print(f"📁 출력 디렉토리: {base_output_dir}")
    print(f"📅 처리 월: {months}")
    print(f"⚙️ 옵션: {options}")
    
    successful = 0
    failed = 0
    
    for month in months:
        success = process_single_file(
            grouper, year, month, base_input_dir, base_output_dir, **options
        )
        
        if success:
            successful += 1
        else:
            failed += 1
    
    # 최종 결과 출력
    print(f"\n{'='*60}")
    print(f"🎉 {year}년 배치 처리 완료!")
    print(f"✅ 성공: {successful}개 파일")
    print(f"❌ 실패: {failed}개 파일")
    print(f"📊 성공률: {successful/(successful+failed)*100:.1f}%" if (successful+failed) > 0 else "0%")
    print(f"{'='*60}")


def process_multiple_years(years: List[int], months: List[int] = None, **options):
    """다년도 배치 처리"""
    print(f"🎯 다년도 배치 처리 시작: {years}")
    
    for year in years:
        try:
            process_year_batch(year, months, **options)
        except Exception as e:
            print(f"❌ {year}년 처리 중 전체 오류 발생: {e}")
    
    print(f"🏁 모든 년도 처리 완료: {years}")


# 사용 예시 및 메인 실행부
def main():
    """메인 실행 함수"""
    grouper = KeywordGrouper()
    
    # 샘플 데이터 생성 (테스트용)
    sample_data = {
        "year": 2024,
        "month": 1,
        "period": "2024-01-01 ~ 2024-01-31",
        "total_articles": 487,
        "keywords": [
            {
                "phrase": "시험 발사",
                "doc_count": 51,
                "score": 18.9,
                "examples": ["예시1", "예시2", "예시3"]
            },
            {
                "phrase": "탄도미사일 발사",
                "doc_count": 40,
                "score": 14.5,
                "examples": ["예시4", "예시5", "예시6"]
            },
            {
                "phrase": "순항미사일 발사",
                "doc_count": 28,
                "score": 9.7,
                "examples": ["예시7", "예시8", "예시9"]
            },
            {
                "phrase": "군사 협력",
                "doc_count": 27,
                "score": 9.3,
                "examples": ["예시10", "예시11", "예시12"]
            },
            {
                "phrase": "해상 사격",
                "doc_count": 25,
                "score": 8.5,
                "examples": ["예시13", "예시14", "예시15"]
            }
        ]
    }
    
    print("=== 자동화된 키워드 그룹화 시스템 ===")
    print("📝 변경 사항:")
    print("- ❌ important_keywords 제거")
    print("- ❌ keyword_bonus 제거") 
    print("- ✅ 자카드(70%) + 편집거리(30%) 유사도만 사용")
    print("- 🚀 자동화된 배치 처리 기능 추가")
    print()
    
    # print("🔧 사용법:")
    # print("1. 단일 년도 처리:")
    # print("   process_year_batch(2024)")
    # print()
    # print("2. 특정 월만 처리:")
    # print("   process_year_batch(2024, [1, 2, 3])")
    # print()
    # print("3. 다년도 처리:")
    # print("   process_multiple_years([2023, 2024])")
    # print()
    # print("4. 사용자 정의 옵션:")
    # print("   process_year_batch(2024,")
    # print("       similarity_threshold=0.3,")
    # print("       min_group_size=2,")
    # print("       max_groups=12,")
    # print("       input_dir='/custom/input/path',")
    # print("       output_dir='/custom/output/path')")
    # print()
    
    return grouper, sample_data


if __name__ == "__main__":
    grouper, sample_data = main()
    
    # 🎯 실제 처리 시작 - 아래 중 하나를 선택하여 사용하세요
    
    # 방법 1: 2024년 전체 처리 (1월~12월)
    process_year_batch(
        year=2025,
        similarity_threshold=0.4,
        min_group_size=2,
        max_groups=12
    )
    
    # 방법 2: 2023년 전체 처리 (1월~12월) - 주석 해제하여 사용
    # process_year_batch(
    #     year=2023,
    #     similarity_threshold=0.4,
    #     min_group_size=2,
    #     max_groups=12
    # )
    
    # 방법 3: 2023년과 2024년 모두 처리 - 주석 해제하여 사용
    # process_multiple_years(
    #     years=[2023, 2024],
    #     similarity_threshold=0.4,
    #     min_group_size=2,
    #     max_groups=12
    # )
    
    # 방법 4: 특정 월만 처리 - 주석 해제하여 사용
    # process_year_batch(
    #     year=2024,
    #     months=[10, 11, 12],  # 10월, 11월, 12월만 처리
    #     similarity_threshold=0.4,
    #     min_group_size=2,
    #     max_groups=12
    # )
    
    # 방법 5: 사용자 정의 경로로 처리 - 주석 해제하여 사용
    # process_year_batch(
    #     year=2024,
    #     input_dir='/custom/input/directory',
    #     output_dir='/custom/output/directory',
    #     similarity_threshold=0.4,
    #     min_group_size=2,
    #     max_groups=12
    # )

=== 자동화된 키워드 그룹화 시스템 ===
📝 변경 사항:
- ❌ important_keywords 제거
- ❌ keyword_bonus 제거
- ✅ 자카드(70%) + 편집거리(30%) 유사도만 사용
- 🚀 자동화된 배치 처리 기능 추가

🚀 2025년 키워드 그룹화 배치 처리 시작
📂 입력 디렉토리: /home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/re_monthly_results
📁 출력 디렉토리: /home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/re_monthly_results_cluster
📅 처리 월: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
⚙️ 옵션: {'similarity_threshold': 0.4, 'min_group_size': 2, 'max_groups': 12}

처리 중: 2025년 1월
입력 파일: /home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/re_monthly_results/2025_01_keywords.json
출력 파일: /home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/re_monthly_results_cluster/2025_01_keyword_grouped.json
=== 자동 키워드 그룹화 시작 (키워드 보너스 제거 버전) ===
원본 키워드 수: 30
유사도 임계값: 0.4
최소 그룹 크기: 2
유사도 계산: 자카드(70%) + 편집거리(30%)

=== 그룹화 완료 ===
최종 키워드 그룹 수: 12
압축률: 60.0%

=== 상위 그룹들 (순수 문자열 유사도 기반) ===
1. 탄도미사일 발사 (점수: 60.3)
   - 통합된 키워드 수: 9
   - 평균 유사도: 0.483
   - 통합 키워드: 탄도미사일 발사, 북한 탄도미사일 발사, 미사일 발사, 단거리 탄도미사일 발사, 탄도미사일 시험 발사, 거리 탄도미사일 시험 발

In [ ]:
# # 이벤트 키워드 추출하는 핵심 로직

# import json
# import os
# import re
# from tqdm import tqdm
# from collections import Counter, defaultdict
# from datetime import datetime
# from konlpy.tag import Okt
# import numpy as np
# from sklearn.feature_extraction.text import TfidfVectorizer
# import joblib

# # =========================
# # 설정
# # =========================
# file_path = '/home/ds4_sia_nolb/#FINAL_POLARIS/03_Bert_summarization/final_preprocessing_data/final_preprocessing.json'
# output_file_path = '/home/ds4_sia_nolb/#FINAL_POLARIS/04_Event_top10/top10_file.json'
# # TF-IDF 벡터라이저 불러오기 및 저장 경로 (전체 코퍼스 기반으로 변경)
# TFIDF_VECTORIZER_PATH = '/home/ds4_sia_nolb/#FINAL_POLARIS/04_Event_top10/idf_vectorizer_for_all_corpus.pkl'

# # =========================
# # 형태소 분석기
# # =========================
# okt = Okt()

# # =========================
# # 날짜 파서 (여러 포맷 허용)
# # =========================
# def parse_date_flexible(s: str):
#     if not s or not isinstance(s, str):
#         return None
#     s = s.strip()

#     candidates = [s]
#     if "T" in s:
#         candidates.append(s[:19])
#         candidates.append(s[:10])
#     if len(s) >= 10:
#         candidates.append(s[:10])
#     if "-" not in s and "." not in s and "/" not in s and len(s) == 8:
#         candidates.append(f"{s[:4]}-{s[4:6]}-{s[6:8]}")

#     fmts = [
#         "%Y-%m-%d",
#         "%Y-%m-%d %H:%M:%S",
#         "%Y-%m-%d %H:%M",
#         "%Y/%m/%d",
#         "%Y/%m/%d %H:%M:%S",
#         "%Y.%m.%d",
#         "%Y.%m.%d %H:%M:%S",
#         "%Y.%m.%d %H:%M",
#         "%Y%m%d",
#         "%Y-%m-%dT%H:%M:%S",
#     ]

#     for cand in candidates:
#         for fmt in fmts:
#             try:
#                 return datetime.strptime(cand, fmt)
#             except Exception:
#                 pass
#     return None

# def parse_date(date_str: str) -> datetime:
#     d = parse_date_flexible(date_str)
#     if d is None:
#         raise ValueError(f"날짜 형식이 올바르지 않습니다: {date_str}")
#     return d

# def extract_pubdate(article):
#     keys = ["pubDate", "pubdate", "time", "date", "published", "pub_date"]
#     for k in keys:
#         if k in article and article[k]:
#             dt = parse_date_flexible(str(article[k]))
#             if dt:
#                 return dt
#     meta = article.get("metadata", {}) or {}
#     for k in keys:
#         if k in meta and meta[k]:
#             dt = parse_date_flexible(str(meta[k]))
#             if dt:
#                 return dt
#     return None

# # =========================
# # 기사 로드 및 필터링 유틸
# # =========================
# def load_all_articles(file_path):
#     try:
#         with open(file_path, 'r', encoding='utf-8') as f:
#             data = json.load(f)
#         if not isinstance(data, list):
#             raise ValueError("JSON 루트는 list 여야 합니다.")
#         return data
#     except FileNotFoundError:
#         print(f"파일을 찾을 수 없습니다: {file_path}")
#         return None
#     except json.JSONDecodeError as e:
#         print(f"JSON 파싱 오류: {e}")
#         return None
#     except Exception as e:
#         print(f"알 수 없는 오류 발생: {e}")
#         return None

# def filter_articles_by_period(articles, start_date_str, end_date_str):
#     sdt = parse_date(start_date_str)
#     edt = parse_date(end_date_str)
    
#     period_articles = []
    
#     all_articles_count = len(articles)
    
#     for article in tqdm(articles, desc=f"기사 처리 중 ({sdt.date()}~{edt.date()})"):
#         pub_date = extract_pubdate(article)
#         if pub_date and sdt <= pub_date <= edt:
#             period_articles.append(article)
    
#     print(f"총 {all_articles_count}개의 기사 중 지정 기간 내 기사 {len(period_articles)}개를 찾았습니다.")
    
#     return period_articles

# # =========================
# # 텍스트 정규화 (표기 통일 약간)
# # =========================
# def normalize_text(t: str) -> str:
#     if not t:
#         return ""
#     t = t.replace("탄도 미사일", "탄도미사일")
#     t = t.replace("순항 미사일", "순항미사일")
#     t = t.replace("극초 음속", "극초음속")
#     t = t.replace("초대형 방사포", "초대형방사포")
#     return t

# # =========================
# # 불용어 & 뉴스 노이즈
# # =========================
# BASE_STOP = set([
#     '가','간','같은','같이','것','게다가','결국','곧','관하여','관련','관한','그','그것','그녀','그들',
#     '그리고','그때','그래','그래서','그러나','그러므로','그러한','그런','그렇게','그외','근거로','기타',
#     '까지도','까지','나','남들','너','누구','다','다가','다른','다만','다소','다수','다시','다음','단','단지',
#     '당신','대','대해서','더군다나','더구나','더라도','더욱이','도','도로','또','또는','또한','때','때문',
#     '라도','라면','라는','로','로부터','로써','를','마저','마치','만약','만일','만큼','모두','무엇','무슨',
#     '무척','물론','및','밖에','바로','보다','뿐이다','사람','사실은','상대적으로','생각','설령','소위','수',
#     '수준','쉽게','시대','시작하여','실로','실제','아니','아무','아무도','아무리','아마도','아울러','아직',
#     '앞에서','앞으로','어느','어떤','어떻게','어디','언제','얼마나','여기','여부','역시','예','오히려',
#     '와','왜','외에도','요','우리','우선','원래','위해서','으로','으로부터','으로써','을','의','의거하여',
#     '의지하여','의해','의해서','의하여','이','이것','이곳','이때','이라고','이러한','이런','이렇게','이제',
#     '이지만','이후','이상','이다','이전','인','일','일단','일반적으로','임시로','입장에서','자','자기','자신',
#     '잠시','저','저것','저기','저쪽','저희','전부','전혀','점에서','정도','제','조금','좀','주로','주제','즉',
#     '즉시','지금','진짜로','차라리','참','참으로','첫번째로','최고','최대','최소','최신','최초','통하여',
#     '통해서','평가','포함한','포함하여','하지만','하면서','하여','한','한때','한번','할','할것이다','할수있다',
#     '함께','해도', '돼다', '서다', '대해', '나오다', '통해', '맞다', 
# ])

# NEWS_STOP = {"기자","연합뉴스","사진","속보","종합","자료","영상","단독","전문","인터뷰","브리핑"}

# # =========================
# # 엔터티 노이즈
# # =========================
# ENTITY_NOISE = {
#     "북한","한국","대한민국","남한","미국","중국","일본","러시아","우크라이나","유엔","나토","NATO","EU","유럽연합",
#     "푸틴","블라디미르 푸틴","바이든","조 바이든","시진핑","김정은","김여정","문재인","윤석열","쇼이구","젤렌스키","통신","중앙","보도"
# }

# # =========================
# # 토큰/텍스트
# # =========================
# def pos_tokens(text: str):
#     text = normalize_text(text or "")
#     return okt.pos(text, norm=True, stem=True)

# def doc_text(a) -> str:
#     return normalize_text(f"{a.get('title','')} {a.get('summary','')}")

# def tokenizer_for_vectorizer(s: str):
#     toks = []
#     for w, t in okt.pos(s, norm=True, stem=True):
#         if t not in ("Noun", "Verb"):
#             continue
#         if len(w) <= 1:
#             continue
#         if w in BASE_STOP or w in NEWS_STOP:
#             continue
#         if w.isdigit():
#             continue
#         toks.append(w)
#     return toks

# # =========================
# # 자동 학습: '행동 동사'와 '행위 명사'
# # =========================
# def learn_action_lexicons(articles, min_df_ratio_verbs=0.002, min_df_ratio_nouns=0.002):
#     verb_doc_df = Counter()
#     action_noun_df = Counter()
#     N_docs = len(articles)

#     for a in tqdm(articles, desc="행동 동사/행위명사 학습 중"):
#         title = a.get('title','') or ''
#         summary = a.get('summary','') or ''
#         p = pos_tokens(f"{title} {summary}")

#         verbs_in_doc = set()
#         action_nouns_in_doc = set()

#         for i, (w, t) in enumerate(p):
#             if t == "Verb":
#                 verbs_in_doc.add(w)
#             if t == "Noun":
#                 ahead = [p[j][0] for j in range(i+1, min(i+3, len(p)))]
#                 if "하다" in ahead or "되다" in ahead:
#                     if w not in BASE_STOP and len(w) > 1:
#                         action_nouns_in_doc.add(w)

#         for v in verbs_in_doc:
#             verb_doc_df[v] += 1
#         for n in action_nouns_in_doc:
#             action_noun_df[n] += 1

#     min_df_verbs = max(5, int(N_docs * min_df_ratio_verbs))
#     min_df_nouns = max(5, int(N_docs * min_df_ratio_nouns))

#     drop_verbs = {"하다","되다","이다","있다"}
#     verb_set = {v for v,df in verb_doc_df.items() if df >= min_df_verbs and v not in drop_verbs}
#     action_nouns = {n for n,df in action_noun_df.items() if df >= min_df_nouns}

#     print(f"학습 결과: 행동동사 {len(verb_set)}개, 행위명사 {len(action_nouns)}개")
#     return verb_set, action_nouns

# def nominalize_verb(v: str) -> str:
#     if v.endswith("하다"):
#         return v[:-2]
#     if v.endswith("되다"):
#         return v[:-2]
#     return v

# # =========================
# # 사건 구 후보 생성 + TF-IDF 결합 랭킹
# # =========================
# def extract_event_phrases_auto(articles, top_k=20, vectorizer=None):
#     N = len(articles)
#     if N == 0:
#         print("⚠ 지정 기간에 기사가 없습니다.")
#         return []

#     print(f"\n지정 기간 내 기사 수: {N}개")
    
#     verb_set, action_nouns = learn_action_lexicons(articles)

#     phrase_df = Counter()
#     phrase_examples = defaultdict(list)

#     for a in tqdm(articles, desc="사건 구 자동 추출 중"):
#         title = a.get('title','') or ''
#         summary = a.get('summary','') or ''
#         p = pos_tokens(f"{title} {summary}")
#         phrases_in_doc = set()

#         prev_nouns = []
#         L = len(p)
#         for i, (w, t) in enumerate(p):
#             if t == "Noun":
#                 if w not in BASE_STOP and len(w) > 1:
#                     prev_nouns.append(w)
#                     if len(prev_nouns) > 5:
#                         prev_nouns = prev_nouns[-5:]

#             if t == "Verb" and w in verb_set:
#                 vnom = nominalize_verb(w)
#                 nn = [n for n in reversed(prev_nouns)][:2]
#                 if nn:
#                     phrases_in_doc.add(f"{nn[0]} {vnom}".strip())
#                     if len(nn) >= 2:
#                         phrases_in_doc.add(f"{nn[1]} {nn[0]} {vnom}".strip())
#                 else:
#                     phrases_in_doc.add(vnom.strip())

#             if t == "Noun" and w in action_nouns:
#                 nn = [n for n in reversed(prev_nouns) if n != w][:2]
#                 base = w
#                 if nn:
#                     phrases_in_doc.add(f"{nn[0]} {base}".strip())
#                     if len(nn) >= 2:
#                         phrases_in_doc.add(f"{nn[1]} {nn[0]} {base}".strip())
#                 else:
#                     phrases_in_doc.add(base.strip())

#                 if i+1 < L and p[i+1][1] == "Noun" and p[i+1][0] in action_nouns:
#                     tail = p[i+1][0]
#                     if nn:
#                         phrases_in_doc.add(f"{nn[0]} {base} {tail}".strip())
#                         if len(nn) >= 2:
#                             phrases_in_doc.add(f"{nn[1]} {nn[0]} {base} {tail}".strip())
#                     else:
#                         phrases_in_doc.add(f"{base} {tail}".strip())

#         cleaned = set()
#         for ph in phrases_in_doc:
#             ph = re.sub(r"\s+", " ", ph).strip()
#             if len(ph.split()) == 1 and len(ph) <= 2:
#                 continue
#             cleaned.add(ph)

#         for ph in cleaned:
#             phrase_df[ph] += 1
#             if len(phrase_examples[ph]) < 3 and title:
#                 phrase_examples[ph].append(title)

#     if vectorizer is None:
#         print("[오류] TfidfVectorizer 객체가 전달되지 않았습니다.")
#         return []

#     corpus_period = [doc_text(a) for a in articles]
#     Xp = vectorizer.transform(corpus_period)
#     tfidf_avg = np.asarray(Xp.mean(axis=0)).ravel()
#     terms = vectorizer.get_feature_names_out()
#     tfidf_dict = {terms[i]: float(tfidf_avg[i]) for i in np.where(tfidf_avg > 0)[0]}

#     print(f"TF-IDF 코퍼스: {len(corpus_period)}개 문서")
#     print(f"TF-IDF 용어수: {len(terms)}")

#     def is_entity_only(ph: str) -> bool:
#         toks = ph.split()
#         if len(toks) <= 2 and any(ent in ph for ent in ENTITY_NOISE):
#             return True
#         ent_hits = sum(1 for t in toks if any(ent in t for ent in ENTITY_NOISE))
#         return (ent_hits >= max(1, len(toks) - 1))

#     def generic_penalty(ph: str) -> int:
#         generic = {"대통령","위원장","정부","당국","관계자","대변인","회의","논의","강조"}
#         return -sum(1 for t in ph.split() if t in generic)

#     def phrase_score(ph: str, df_cnt: int) -> float:
#         tfidf = tfidf_dict.get(ph, 0.0)
#         score = 0.6 * tfidf + 0.4 * float(df_cnt)

#         if is_entity_only(ph):
#             score -= 6.0
#         score += generic_penalty(ph)
#         if len(ph.split()) <= 2:
#             score -= 1.5
#         return score

#     scored = []
#     for ph, cnt in phrase_df.items():
#         if is_entity_only(ph):
#             continue
#         scored.append( (ph, cnt, phrase_score(ph, cnt)) )

#     scored.sort(key=lambda x: (x[2], x[1]), reverse=True)
#     ranked = [(ph, cnt, phrase_examples.get(ph, [])) for ph, cnt, _ in scored[:top_k]]
#     return ranked

# # =========================
# # 전체 코퍼스용 TF-IDF 벡터라이저 사전 학습
# # =========================
# def pre_train_vectorizer(articles, save_path):
#     if os.path.exists(save_path):
#         print(f"✔️ 기존 TF-IDF 벡터라이저 파일 '{save_path}'이 이미 존재합니다. 학습을 건너뜁니다.")
#         return joblib.load(save_path)
    
#     print(f"🔍 전체 코퍼스용 TF-IDF 벡터라이저를 새로 학습합니다.")
    
#     full_corpus = [doc_text(a) for a in articles]
    
#     vectorizer = TfidfVectorizer(
#         tokenizer=tokenizer_for_vectorizer,
#         ngram_range=(1, 3),
#         min_df=5,
#         max_df=0.85,
#         sublinear_tf=True,
#         norm='l2'
#     )
#     vectorizer.fit(full_corpus)
#     joblib.dump(vectorizer, save_path)
#     print(f"✅ 전체 코퍼스 기반 TF-IDF 벡터라이저를 '{save_path}'에 저장했습니다.")
#     return vectorizer

# # =========================
# # 실행부
# # =========================
# if __name__ == '__main__':
#     try:
#         # 1. 전체 기사 데이터를 한 번만 로드합니다.
#         all_articles = load_all_articles(file_path)
#         if not all_articles:
#             print("전체 코퍼스를 로드할 수 없습니다. 프로그램을 종료합니다.")
#             exit()

#         # 2. 전체 코퍼스 데이터를 이용해 TF-IDF 벡터라이저를 학습/로드합니다.
#         vectorizer = pre_train_vectorizer(all_articles, TFIDF_VECTORIZER_PATH)
        
#         # 3. 날짜를 사용자에게 입력받습니다.
#         start = input("시작일 입력 (YYYY-MM-DD 또는 YYYYMMDD): ").strip()
#         end = input("종료일 입력 (YYYY-MM-DD 또는 YYYYMMDD): ").strip()

#         print("\n✔️ 날짜가 입력되었습니다. 지정 기간 내 모든 기사를 기반으로 분석을 시작합니다.")
        
#         # 4. 로드된 전체 데이터에서 지정 기간 기사만 필터링합니다.
#         period_articles = filter_articles_by_period(all_articles, start, end)
        
#         if not period_articles:
#             print("분석할 기사가 없습니다. 프로그램을 종료합니다.")
#         else:
#             # 5. 사건구를 추출하고 랭킹을 매깁니다.
#             events = extract_event_phrases_auto(
#                 period_articles,
#                 top_k=10,
#                 vectorizer=vectorizer
#             )

#             print("\n=== 기간별 '사건 구' TOP 10 (TF-IDF + DF 결합) ===")
#             if not events:
#                 print("해당 기간에 추출된 사건 구가 없습니다.")
#             else:
#                 for i, (ph, cnt, examples) in enumerate(events, 1):
#                     ex_str = " | 예시: " + " / ".join(examples) if examples else ""
#                     print(f"{i}. {ph}   (문서수={cnt}){ex_str}")
    
#     except ValueError as e:
#         print(f"오류: {e}. 올바른 날짜 형식을 입력해주세요. 프로그램을 종료합니다.")
#     except Exception as e:
#         print(f"예기치 않은 오류가 발생했습니다: {e}")

✔️ 기존 TF-IDF 벡터라이저 파일 '/home/ds4_sia_nolb/#FINAL_POLARIS/04_Event_top10/idf_vectorizer_for_all_corpus.pkl'이 이미 존재합니다. 학습을 건너뜁니다.

✔️ 날짜가 입력되었습니다. 지정 기간 내 모든 기사를 기반으로 분석을 시작합니다.


기사 처리 중 (2024-01-01~2024-02-01): 100%|██████████| 80810/80810 [00:01<00:00, 80230.24it/s]


총 80810개의 기사 중 지정 기간 내 기사 510개를 찾았습니다.

지정 기간 내 기사 수: 510개


행동 동사/행위명사 학습 중: 100%|██████████| 510/510 [00:17<00:00, 29.27it/s]


학습 결과: 행동동사 121개, 행위명사 144개


사건 구 자동 추출 중: 100%|██████████| 510/510 [00:14<00:00, 35.58it/s]


TF-IDF 코퍼스: 510개 문서
TF-IDF 용어수: 340845

=== 기간별 '사건 구' TOP 10 (TF-IDF + DF 결합) ===
1. 시험 발사   (문서수=51)
2. 탄도미사일 발사   (문서수=38)
3. 군사 협력   (문서수=27)
4. 순항미사일 발사   (문서수=25)
5. 해상 사격   (문서수=24)
6. 결의 위반   (문서수=23)
7. 미사일 발사   (문서수=22)
8. 우크라이나 공격 사용   (문서수=18)
9. 안보리 결의 위반   (문서수=18)
10. 발사 밝히다   (문서수=21)
